In [31]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from google.colab import files

In [32]:
file_path = '/content/Lithium Carbonate 99%Min China Spot Historical Data (2).csv'
LTCB_99 = pd.read_csv(file_path)

In [33]:
LTCB_99['Date'] = pd.to_datetime(LTCB_99['Date'])
LTCB_99 = LTCB_99.sort_values('Date')
LTCB_99.set_index('Date', inplace=True)

LTCB_99.rename(columns = {'Change %': 'Obs_Return'}, inplace = True)
LTCB_99['Obs_Return'] = (LTCB_99['Obs_Return'].str.replace('%', '').astype(float))/100

In [35]:
return_na_count = LTCB_99['Obs_Return'].isna().sum()
if return_na_count > 0:
  LTCB_99 = LTCB_99.dropna(subset=['Obs_Return'])
  print("Number of missing values:", return_na_count)

In [37]:
print(LTCB_99.head())

                 Price        Open        High         Low  Vol.  Obs_Return
Date                                                                        
2017-05-10  123,000.00  123,000.00  127,000.00  119,000.00   NaN         0.0
2017-05-11  123,000.00  123,000.00  127,000.00  119,000.00   NaN         0.0
2017-05-12  123,000.00  123,000.00  127,000.00  119,000.00   NaN         0.0
2017-05-15  123,000.00  123,000.00  127,000.00  119,000.00   NaN         0.0
2017-05-16  123,000.00  123,000.00  127,000.00  119,000.00   NaN         0.0


In [38]:
def unsmooth_returns_1_step(observed_returns, max_lags):
    """
    Unsmooth observed returns using a 1-step MA Unsmoothing Method.

    Parameters:
    - observed_returns: observed (smoothed) returns.
    - max_lags: Maximum number of lags (H) to consider for the MA model.

    Returns:
    - true_returns: estimated true (unsmoothed) returns.
    """

    # Fit an MA model to the observed returns
    # Using AIC to choose the best number of lags
    best_aic = np.inf
    best_order = None
    best_model = None

    for lag in range(max_lags + 1):
        try:
            model = sm.tsa.ARIMA(observed_returns, order=(0, 0, lag)).fit()
            if model.aic < best_aic:
                best_aic = model.aic
                best_order = lag
                best_model = model
        except:
            continue
    # Extract the residuals from the best MA model
    if best_model is not None:
        true_returns = best_model.resid + observed_returns.mean()
    else:
        # If no model is successfully fitted, fallback to observed returns
        true_returns = observed_returns.copy()

    return true_returns

In [39]:
one_step_true_MA = unsmooth_returns_1_step(observed_returns=LTCB_99['Obs_Return'], max_lags=10)

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


In [41]:
one_step_true_MA.to_csv(r'LTCB99_1_step_true_MA_APR11.csv', index=True, header=True)
files.download('LTCB99_1_step_true_MA_APR11.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>